# Topic Modeling with BERTopic

english is fine, no need for multilingual.
recenzioni lunghe = ragiungono tanti, poco veritiere, molto influenti 

topic_model.merge_topics(corpus, topics_to_merge=[])

In [208]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sentence_transformers import SentenceTransformer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab',quiet=True)
nltk.download('punkt',quiet=True)
pass

In [270]:
df = pd.read_csv('../1-etl/reviews_clean.csv').fillna('')
corpus = df[df['stars']<=5]['content']
corpus = list(filter(lambda x:len(x)>20,corpus))
corpus = [sent for doc in corpus for sent in sent_tokenize(doc)]
len(corpus)

2821

In [273]:
# Create embeddings from the documents
sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(list(corpus))

In [ ]:
# Define sub-models
vectorizer = CountVectorizer(stop_words="english")
umap_model = UMAP(n_neighbors=5, n_components=10,
                  min_dist=0.0, metric='cosine')
# hdbscan_model = HDBSCAN(min_cluster_size=10, min_samples=2, metric='euclidean')

# Train our topic model with BERTopic
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    # embedding_model=sentence_model,
    # umap_model=umap_model,
    # hdbscan_model=hdbscan_model,
    # vectorizer_model=vectorizer,
    n_gram_range=(1, 2)
)
topics, probs = topic_model.fit_transform(corpus, embeddings)
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,1161,-1_the_to_it_and,"the, to, it","[the, to, it, and, printer, this, that, of, fo...","[Constantly ""offline"", disconnecting from my r..."
1,0,124,0_hp_with_support_with hp,"hp, with, support","[hp, with, support, with hp, hp and, and, amaz...",[I spent literal hours on the laptop with hp s...
2,1,83,1_service_customer_customer service_he,"service, customer, customer servic","[service, customer, customer service, he, me, ...",[When calling customer support they immediatel...
3,2,77,2_set_easy_set up_up,"set, easy, set up","[set, easy, set up, up, to set, setup, printer...",[This printer was vry easy to set up and conne...
4,3,74,3_quality_print quality_quality is_prints,"quality, print quality, quality is","[quality, print quality, quality is, prints, t...",[The print quality is not very good on this on...
5,4,74,4_tray_paper_paper tray_the paper,"tray, paper, paper tray","[tray, paper, paper tray, the paper, the, slid...",[The only thing I don’t like is the paper tray...
6,5,67,5_hp_hp printer_hp printers_printers,"hp, hp printer, hp printers","[hp, hp printer, hp printers, printers, years,...",[I bought this HP printer to replace a very si...
7,6,62,6_printer_this_printer is_for,"printer, this, printer is","[printer, this, printer is, for, good, this pr...",[Have not learned everything about it but so f...
8,7,49,7_scan_scanning_to scan_sided,"scan, scanning, to scan","[scan, scanning, to scan, sided, page, documen...",[The duplex function is useless: When trying t...
9,8,47,8_buy_would_price_buy it,"buy, would, price","[buy, would, price, buy it, recommend, would n...","[Would definitely buy again., Would definitely..."


In [248]:
topic_model.save('model.bertopic')

2025-01-17 23:54:01,739 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [278]:
topic_model = BERTopic.load('model.bertopic')

In [295]:
print(corpus[13])
topic_model.visualize_distribution(probs[13], min_probability=0.02)

I ASSUMMED (my fault) that if i didn't want the instant ink auto delivered that i could just buy ink as needed.


In [275]:
topic_model.visualize_topics()

In [237]:
topic_distr, topic_token_distr = topic_model.approximate_distribution(
    corpus, calculate_tokens=True)
topic_model.visualize_approximate_distribution(corpus[1], topic_token_distr[1])

100%|██████████| 3/3 [00:00<00:00,  7.67it/s]


,It,stopped,working,and,showed,me,message,to,set,up,my,printer
2_set_easy_set up_up,0.000,0.000,0.000,0.000,0.000,0.113,0.337,0.598,0.799,0.687,0.462,0.201
9_set up_set_easy_up,0.000,0.000,0.000,0.000,0.000,0.139,0.436,0.802,1.064,0.925,0.628,0.262
14_return it_it_return_could return,0.126,0.126,0.126,0.126,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
20_wireless_wifi_to_phone,0.000,0.000,0.000,0.000,0.000,0.000,0.138,0.300,0.300,0.300,0.162,0.000
21_ink_instant ink_instant_my printer,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.102,0.102,0.102,0.102


In [296]:
topic_model.visualize_heatmap( width=850, height=850)

In [297]:
topic_labels = topic_model.generate_topic_labels(
    nr_words=3,
    topic_prefix=False,
    word_length=15,
    separator=", ")
topic_model.set_topic_labels(topic_labels)

In [298]:
topic_model.visualize_documents(
    corpus,
    embeddings=embeddings,
    hide_annotations=False,
    topics=range(35),
    custom_labels=True,
    width=2000,
    height=1200,
)